In [86]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from implementations import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [89]:
DATA_TRAIN_PATH = 'data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [110]:
tX = tX[:10000][:].copy()
y = y[:10000][:].copy()[:, np.newaxis]

In [111]:
mean = np.mean(tX, axis=0)
std = np.std(tX, axis=0)
tX = (tX - mean) / std

In [112]:
tX_aug = np.hstack((np.ones((tX.shape[0], 1)), tX))
tX_aug.shape

(10000, 31)

In [121]:
w = np.random.rand(tX.shape[1] + 1, 1)
w.shape
w

array([[0.7621476 ],
       [0.58778864],
       [0.38079447],
       [0.54476012],
       [0.37978465],
       [0.11668309],
       [0.06332338],
       [0.8973591 ],
       [0.454953  ],
       [0.32577985],
       [0.12797609],
       [0.11766762],
       [0.62077404],
       [0.94057834],
       [0.91034117],
       [0.12044248],
       [0.65421827],
       [0.0380134 ],
       [0.59505263],
       [0.73961728],
       [0.37971818],
       [0.07481441],
       [0.52726832],
       [0.28518496],
       [0.86244383],
       [0.76984354],
       [0.66489981],
       [0.90017901],
       [0.31382793],
       [0.094757  ],
       [0.61702997]])

In [118]:
w_final, loss = logistic_regression(y, tX_aug, w, 10000, 0.00001)
loss

Iteration : 0 with loss [[25226.85920863]]
Iteration : 10 with loss [[11722.20442694]]
Iteration : 20 with loss [[1197.35403617]]
Iteration : 30 with loss [[-5796.75005835]]
Iteration : 40 with loss [[-10993.76119501]]
Iteration : 50 with loss [[-15328.69471631]]
Iteration : 60 with loss [[-19228.42986819]]
Iteration : 70 with loss [[-22894.67131202]]
Iteration : 80 with loss [[-26424.81870017]]
Iteration : 90 with loss [[-29868.83488485]]
Iteration : 100 with loss [[-33254.6436323]]
Iteration : 110 with loss [[-36599.29270374]]
Iteration : 120 with loss [[-39913.69190558]]
Iteration : 130 with loss [[-43205.12646928]]
Iteration : 140 with loss [[-46478.50570884]]
Iteration : 150 with loss [[-49733.48755083]]
Iteration : 160 with loss [[-52972.77425282]]
Iteration : 170 with loss [[-56202.38755794]]
Iteration : 180 with loss [[-59424.08823861]]
Iteration : 190 with loss [[-62639.06826358]]
Iteration : 200 with loss [[-65848.27649916]]
Iteration : 210 with loss [[-69052.4831755]]
Iterat

/home/flo/EPFL/ML/ML-Project-01/implementations.py:180: RuntimeWarning: overflow encountered in exp
  #print(w)
/home/flo/EPFL/ML/ML-Project-01/implementations.py:163: RuntimeWarning: overflow encountered in exp
  sigmoid_t = (np.exp(t)) / (1 + np.exp(t))
/home/flo/EPFL/ML/ML-Project-01/implementations.py:163: RuntimeWarning: invalid value encountered in true_divide
  sigmoid_t = (np.exp(t)) / (1 + np.exp(t))


Iteration : 5380 with loss [[nan]]
Iteration : 5390 with loss [[nan]]
Iteration : 5400 with loss [[nan]]
Iteration : 5410 with loss [[nan]]
Iteration : 5420 with loss [[nan]]
Iteration : 5430 with loss [[nan]]
Iteration : 5440 with loss [[nan]]
Iteration : 5450 with loss [[nan]]
Iteration : 5460 with loss [[nan]]
Iteration : 5470 with loss [[nan]]
Iteration : 5480 with loss [[nan]]
Iteration : 5490 with loss [[nan]]
Iteration : 5500 with loss [[nan]]
Iteration : 5510 with loss [[nan]]
Iteration : 5520 with loss [[nan]]
Iteration : 5530 with loss [[nan]]
Iteration : 5540 with loss [[nan]]
Iteration : 5550 with loss [[nan]]
Iteration : 5560 with loss [[nan]]
Iteration : 5570 with loss [[nan]]
Iteration : 5580 with loss [[nan]]
Iteration : 5590 with loss [[nan]]
Iteration : 5600 with loss [[nan]]
Iteration : 5610 with loss [[nan]]
Iteration : 5620 with loss [[nan]]
Iteration : 5630 with loss [[nan]]
Iteration : 5640 with loss [[nan]]
Iteration : 5650 with loss [[nan]]
Iteration : 5660 wit

array([[nan]])

In [124]:
w = np.random.rand(tX.shape[1] + 1, 1)
w.shape
w
loss, w = reg_logistic_regression(y, tX_aug, 0.8, w, 1000, 0.00001, True)

Iteration : 0 with loss [[20612.03550681]]
Iteration : 10 with loss [[6489.98496497]]
Iteration : 20 with loss [[-3550.4068557]]
Iteration : 30 with loss [[-9205.0446913]]
Iteration : 40 with loss [[-13591.06092194]]
Iteration : 50 with loss [[-17507.23345623]]
Iteration : 60 with loss [[-21181.85950315]]
Iteration : 70 with loss [[-24714.11582663]]
Iteration : 80 with loss [[-28149.45209814]]
Iteration : 90 with loss [[-31523.23034507]]
Iteration : 100 with loss [[-34854.16837925]]
Iteration : 110 with loss [[-38153.26412587]]
Iteration : 120 with loss [[-41427.83854336]]
Iteration : 130 with loss [[-44682.98265643]]
Iteration : 140 with loss [[-47922.36254775]]
Iteration : 150 with loss [[-51148.69619427]]
Iteration : 160 with loss [[-54364.04880562]]
Iteration : 170 with loss [[-57570.02276977]]
Iteration : 180 with loss [[-60767.88380434]]
Iteration : 190 with loss [[-63958.64714166]]
Iteration : 200 with loss [[-67143.13793504]]
Iteration : 210 with loss [[-70322.03459787]]
Iterat

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)